In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import xml.etree.ElementTree as ET
import os
from tqdm import tqdm

class Reader:
    def __init__(self, voc_root):
        """Pascal VOC format data reader

        Args:
            voc_root (str) - pascal voc format data root

        """
        self.voc_root = Path(voc_root)
        self.xml_root = self.voc_root #/ 'Annotations'
        self.txt_root = self.voc_root #/ 'ImageSets' / 'Main'
        self.img_root = self.voc_root #/ 'JPEGImages'
        self.df = pd.DataFrame(self.parse_xml(self.xml_root))
        self.df['img_path'] = self.df.stem.map(lambda x: self.img_root / (x + '.jpg'))
        train_array, val_array, train_val_array, test_array = self.parse_txt(self.txt_root)
        self.df['train'] = self.df.stem.isin(train_array)
        self.df['val'] = self.df.stem.isin(val_array)
        self.df['trainval'] = self.df.stem.isin(train_val_array)
        self.df['test'] = self.df.stem.isin(test_array)

    def parse_xml(self, xml_root):
        ret = []
        for i in tqdm(xml_root.glob('*.xml')):
            ret.extend(self.parse_single_xml(i))
        return ret

    def parse_single_xml(self, xml_path):
        ret = []

        tree = ET.parse(str(xml_path))
        root = tree.getroot()
        size = root.find('size')
        if size:
            width = int(size.find('width').text)
            height = int(size.find('height').text)
            depth = int(size.find('depth').text)
        else:
            width, height, depth = -1, -1, -1

        for obj in root.findall('object'):
            name = obj.find('name').text
            difficult = int(obj.find('difficult').text)
            bnd_box = obj.find('bndbox')
            bbox = [
                int(float(bnd_box.find('xmin').text)),
                int(float(bnd_box.find('ymin').text)),
                int(float(bnd_box.find('xmax').text)),
                int(float(bnd_box.find('ymax').text))
            ]
            h, w = bbox[3]-bbox[1], bbox[2]-bbox[0]
            ratio = h / w
            ret.append(dict(name=name, difficult=difficult,
                x1=bbox[0], y1=bbox[1], x2=bbox[2], y2=bbox[3],
                xml_path=xml_path, stem=xml_path.stem,
                width=width, height=height, depth=depth,
                h=h, w=w, ratio=ratio))
        return ret

    def parse_txt(self, txt_root):
        train_array=self.parse_single_txt(txt_root / 'train.txt')
        test_array=self.parse_single_txt(txt_root / 'test.txt')
        val_array=self.parse_single_txt(txt_root / 'val.txt')
        train_val_array=self.parse_single_txt(txt_root / 'trainval.txt')
        return train_array, val_array, train_val_array, test_array

    def parse_single_txt(self, txt_path):
        if not os.path.exists(txt_path):
            return []
        with open(txt_path) as f:
            return list(f.readlines())

    def show(self):
        import cv2
        grouped = self.df.groupby('stem')
        # length = self.df.
        length = self.df.stem.unique().size
        indexes = self.df.stem.unique()
        def nothing(x):
            index = indexes[x]
            sub_table = grouped.get_group(index)
            img_path = str(sub_table.iloc[0]['img_path'])
            img = cv2.imread(img_path)
            for index, row in sub_table.iterrows():
                cv2.rectangle(img, (row['x1'], row['y1']), (row['x2'], row['y2']),
                              (0, 255, 0))
            # img = cv2.resize(img, (800, 800))
            cv2.imshow('image_win', img)
            # cv2.waitKey()

        cv2.namedWindow('image_win')
        cv2.namedWindow('tracker_win')
        cv2.createTrackbar('tracker', 'tracker_win', 0, length - 1, nothing)

        if cv2.waitKey() == 'q':
            cv2.destroyAllWindows()


## Read XML in folder

In [2]:
reader = Reader("../debug_images/")
reader.df

27it [00:00, 9884.46it/s]


,name,difficult,x1,y1,x2,y2,xml_path,stem,width,height,depth,h,w,ratio,img_path,train,val,trainval,test
0,face,0,645,351,818,536,../debug_images/06b81bcf2f8926ce.xml,06b81bcf2f8926ce,1024,891,3,185,173,1.069364,../debug_images/06b81bcf2f8926ce.jpg,False,False,False,False
1,person,0,11,206,843,879,../debug_images/06b81bcf2f8926ce.xml,06b81bcf2f8926ce,1024,891,3,673,832,0.808894,../debug_images/06b81bcf2f8926ce.jpg,False,False,False,False
2,person,0,377,383,504,543,../debug_images/09e22dd8a7a408c7.xml,09e22dd8a7a408c7,683,1024,3,160,127,1.259843,../debug_images/09e22dd8a7a408c7.jpg,False,False,False,False
3,person,0,609,389,683,598,../debug_images/09e22dd8a7a408c7.xml,09e22dd8a7a408c7,683,1024,3,209,74,2.824324,../debug_images/09e22dd8a7a408c7.jpg,False,False,False,False
4,face,0,383,408,432,476,../debug_images/09e22dd8a7a408c7.xml,09e22dd8a7a408c7,683,1024,3,68,49,1.387755,../debug_images/09e22dd8a7a408c7.jpg,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,face,0,215,71,229,91,../debug_images/0bf36835f38ef50b.xml,0bf36835f38ef50b,1024,575,3,20,14,1.428571,../debug_images/0bf36835f38ef50b.jpg,False,False,False,False
112,face,0,292,13,314,46,../debug_images/0bf36835f38ef50b.xml,0bf36835f38ef50b,1024,575,3,33,22,1.500000,../debug_images/0bf36835f38ef50b.jpg,False,False,False,False
113,person,0,622,324,752,435,../debug_images/00d458259fe8a694.xml,00d458259fe8a694,1024,683,3,111,130,0.853846,../debug_images/00d458259fe8a694.jpg,False,False,False,False
114,face,0,406,269,487,339,../debug_images/06eb8da7ead1ca12.xml,06eb8da7ead1ca12,1024,768,3,70,81,0.864198,../debug_images/06eb8da7ead1ca12.jpg,False,False,False,False


## To fastdup format

In [3]:
annots_df = reader.df
annots_df = annots_df[['name', 'x1', 'y1', 'x2', 'y2', 'img_path']]
annots_df.rename(columns={'name':'label', 'img_path':'filename'}, inplace=True)

# Convert bounding boxes
annots_df['col_x'] = (annots_df['x1'] + annots_df['x2']) / 2
annots_df['row_y'] = (annots_df['y1'] + annots_df['y2']) / 2
annots_df['width'] = annots_df['x2'] - annots_df['x1']
annots_df['height'] = annots_df['y2'] - annots_df['y1']

annots_df = annots_df[['filename', 'col_x', 'row_y', 'width', 'height', 'label']]

annots_df

/tmp/ipykernel_69431/3602347420.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  annots_df.rename(columns={'name':'label', 'img_path':'filename'}, inplace=True)
/tmp/ipykernel_69431/3602347420.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  annots_df['col_x'] = (annots_df['x1'] + annots_df['x2']) / 2
/tmp/ipykernel_69431/3602347420.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-do

,filename,col_x,row_y,width,height,label
0,../debug_images/06b81bcf2f8926ce.jpg,731.5,443.5,173,185,face
1,../debug_images/06b81bcf2f8926ce.jpg,427.0,542.5,832,673,person
2,../debug_images/09e22dd8a7a408c7.jpg,440.5,463.0,127,160,person
3,../debug_images/09e22dd8a7a408c7.jpg,646.0,493.5,74,209,person
4,../debug_images/09e22dd8a7a408c7.jpg,407.5,442.0,49,68,face
...,...,...,...,...,...,...
111,../debug_images/0bf36835f38ef50b.jpg,222.0,81.0,14,20,face
112,../debug_images/0bf36835f38ef50b.jpg,303.0,29.5,22,33,face
113,../debug_images/00d458259fe8a694.jpg,687.0,379.5,130,111,person
114,../debug_images/06eb8da7ead1ca12.jpg,446.5,304.0,81,70,face


## Merge corrected annotations to previous annotations